<h1><center>MNIST</center></h1>

## 1. benötigte Bibliotheken importieren

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.keras.callbacks import ModelCheckpoint

## 2. Daten einlesen

In [ ]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.mnist.load_data()

## 3. Daten prüfen

In [ ]:
# 1 Bild anzeigen lassen
image_index = 1 # You may select anything up to 60,000
print(y_train[image_index]) # The label is 8
plt.imshow(x_train[image_index], cmap='Greys')
plt.show()

In [ ]:
# Auflösung prüfen

print(x_train.shape)

In [ ]:
# 60000 Bilder vorhanden, jeweils mit einer 28 x 28 Auflösung

## 4. Preprocessing

In [ ]:
# Hier werden die Daten in ein maschinenfreundlicheres Format gebracht
# Da die Daten aber schon schwarz-weiß sind 
# und eine 28x28 Auflösung besitzen fällt dieser Teil in diesem Beispiel fast komplett weg

In [ ]:
# Da die Information über den Farbkanal nicht vorhanden ist, müssen wir sie hinzufügen

In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1) # Die 1. Position gibt an wie viele Bilder verwendet werden.
                                         # -1 steht für eine beliebige Zahl
x_val = x_val.reshape(-1, 28, 28, 1) 

In [ ]:
# Normalisieren

In [ ]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')

In [ ]:
x_train /= 255
x_val /= 255

## 5. Hyperparameter setzen

In [ ]:
epoch = 10 # Anzahl der Durchläufe
num_filters= 10 # Anzahl der Filter
filter_size = 5 # Größe der Filter
pool_size = 5 # Pool Größe
DROP = 0.2 # wie viel % soll "vergessen" werden
lr=0.001 # wie schnell soll gelernt werden
DECAY = 1e-5 # wie schnell soll die Lernkurve abnehmen
opt = tf.keras.optimizers.Adam(lr=0.001, decay=DECAY) # Adam als Optimizer festlegen

## 6. Neuronales Netz bilden

In [ ]:
model = Sequential([
    Conv2D(num_filters, filter_size, input_shape=(28,28, 1), padding="same"),
    Dropout(DROP),
    MaxPooling2D(pool_size=pool_size, padding="same"),
    Conv2D(num_filters, filter_size, padding="same"),
    Dropout(DROP),
    Conv2D(num_filters, filter_size, padding="same"),
    Dropout(DROP),
    MaxPooling2D(pool_size=pool_size, padding="same"),
    Conv2D(num_filters, filter_size, padding="same"),
    Dropout(DROP),
    MaxPooling2D(pool_size=pool_size, padding="same"),
    Conv2D(num_filters, filter_size, padding="same"),
    Dropout(DROP),
    Conv2D(num_filters, filter_size, padding="same"),
    Flatten(),

    Dense(10, activation='softmax'),
])

In [ ]:
# Hierbei muss natürlich nicht jedes Layer gleich viele Filter und Filtergrößen besitzen
# Das letzte Dense Layer hat 10 Neuronen, da es 10 mögliche Features gibt (0-9).

In [ ]:
#Model summary anzeigen lassen für mehr Details
model.summary()

## 7. Model kompilieren

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"]
              )

In [ ]:
# loss gibt die Funktion an, mit der der Loss berechnet wird.

## 8. Speicherorte festlegen

In [ ]:
NAME = f"mnist-{int(time.time())}"
model.save_weights(f'Models_mnist/{NAME}-weights.h5')
tensorboard = TensorBoard(log_dir=f"Logs_mnist/{NAME}")

filepath = f"mnist-num{num_filters}-size{filter_size}-pool{pool_size}-"+"{epoch:02d}-{val_acc:.3f}"  
checkpoint = ModelCheckpoint("Models_mnist/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max'))


## 9. Training starten

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=epoch,
    validation_data=(x_val,y_val),
    callbacks = [tensorboard]
)

## 10. Neuronales Netz optimieren

In [ ]:
# Dieses Neuronale Netz optimieren, indem man Hyperparameter oder die Art und Anzahl der Layer verändert
# 98% Val_Acc möglich
# Viel Spaß